In [2]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numba import jit
import pstats
import heapq
import time
import cProfile
import pandas as pd
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation

puzzle_info_path = 'puzzle_info.csv'
puzzles_path = 'puzzles.csv'
sample_submission_path = 'sample_submission.csv'
my_submission_path = 'submission.csv'

puzzle_info_df = pd.read_csv(puzzle_info_path)
puzzles_df = pd.read_csv(puzzles_path)
sample_submission_df = pd.read_csv(sample_submission_path)
my_submission_df = pd.read_csv(my_submission_path)

In [13]:
puzzles_df[puzzles_df["puzzle_type"] == 'globe_1/8']
# Two types: permutation & colors

,id,puzzle_type,solution_state,initial_state,num_wildcards
338,338,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,I;P;O;A;A;D;F;L;J;M;G;M;P;F;E;J;E;B;O;G;H;D;N;...,0
339,339,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,K;I;C;B;M;J;D;G;E;H;N;E;F;I;D;K;L;P;A;O;B;N;J;...,0
340,340,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,M;O;L;M;P;F;L;B;I;G;E;J;A;G;O;I;C;H;D;P;B;A;N;...,0
341,341,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,P;O;G;J;A;L;F;C;D;B;P;N;F;M;I;G;O;M;H;H;E;N;K;...,0
342,342,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,F;H;O;M;C;J;K;I;J;L;B;N;B;G;K;H;D;P;A;D;E;N;P;...,2
343,343,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N21;N1;N9;N20;N30;N8;N25;N26;N11;N2;N16;N27;N4...,0
344,344,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N30;N0;N22;N15;N24;N19;N14;N18;N31;N6;N23;N16;...,0
345,345,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N19;N11;N22;N29;N15;N7;N14;N16;N0;N18;N9;N24;N...,0
346,346,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N9;N7;N17;N11;N30;N14;N5;N0;N16;N31;N6;N4;N1;N...,0
347,347,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N26;N10;N17;N24;N11;N3;N14;N16;N23;N2;N21;N1;N...,0


In [25]:
idx = 343
N = 1+1
M = int(2*8)

In [64]:
puzzles_df[puzzles_df["puzzle_type"] == "globe_1/8"]

,id,puzzle_type,solution_state,initial_state,num_wildcards
338,338,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,I;P;O;A;A;D;F;L;J;M;G;M;P;F;E;J;E;B;O;G;H;D;N;...,0
339,339,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,K;I;C;B;M;J;D;G;E;H;N;E;F;I;D;K;L;P;A;O;B;N;J;...,0
340,340,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,M;O;L;M;P;F;L;B;I;G;E;J;A;G;O;I;C;H;D;P;B;A;N;...,0
341,341,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,P;O;G;J;A;L;F;C;D;B;P;N;F;M;I;G;O;M;H;H;E;N;K;...,0
342,342,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,F;H;O;M;C;J;K;I;J;L;B;N;B;G;K;H;D;P;A;D;E;N;P;...,2
343,343,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N21;N1;N9;N20;N30;N8;N25;N26;N11;N2;N16;N27;N4...,0
344,344,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N30;N0;N22;N15;N24;N19;N14;N18;N31;N6;N23;N16;...,0
345,345,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N19;N11;N22;N29;N15;N7;N14;N16;N0;N18;N9;N24;N...,0
346,346,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N9;N7;N17;N11;N30;N14;N5;N0;N16;N31;N6;N4;N1;N...,0
347,347,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N26;N10;N17;N24;N11;N3;N14;N16;N23;N2;N21;N1;N...,0


# Solving 2xM

In [72]:
test_globe = np.array([i for i in range(2*M)])
def print_globe(b):
    print(np.asarray(b).reshape(2, M))

r0_perm = np.array([i for i in range(2*M)])
r1_perm = np.array([i for i in range(2*M)])
f0_perm = np.array([i for i in range(2*M)])

r0_perm[M-1] = 0
for i in range(0, M-1):
    r0_perm[i] = i+1

r1_perm[2 * M - 1] = M
for i in range(M, 2*M-1):
    r1_perm[i] = i+1

for i in range(M//2):
    f0_perm[i] = M + M//2 - 1 - i

for i in range(M, M + M//2):
    f0_perm[i] = M//2-1 - (i-M)

nr0_perm = np.array([i for i in range(2*M)])
nr1_perm = np.array([i for i in range(2*M)])

for i in range(M-1):
    nr0_perm = nr0_perm[r0_perm]

for i in range(M-1):
    nr1_perm = nr1_perm[r1_perm]

In [73]:
apply_move = {}

def r0(arr, moves):
    moves.append('r0')
    arr = arr[r0_perm]
    return arr

def nr0(arr, moves):
    moves.append('-r0')
    arr = arr[nr0_perm]
    return arr

def r1(arr, moves):
    moves.append('r1')
    arr = arr[r1_perm]
    return arr

def nr1(arr, moves):
    moves.append('-r1')
    arr = arr[nr1_perm]
    return arr

def f0(arr, moves):
    moves.append('f0')
    arr = arr[f0_perm]
    return arr

apply_move['r0'] = r0
apply_move['-r0'] = nr0
apply_move['r1'] = r1
apply_move['-r1'] = nr1
apply_move['f0'] = f0

print("r0")
print_globe(r0(test_globe, []))
print("nr0")
print_globe(nr0(test_globe, []))
print("r1")
print_globe(r1(test_globe, []))
print("f0")
print_globe(f0(test_globe, []))

r0
[[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15  0]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
nr0
[[15  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
r1
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 16]]
f0
[[23 22 21 20 19 18 17 16  8  9 10 11 12 13 14 15]
 [ 7  6  5  4  3  2  1  0 24 25 26 27 28 29 30 31]]


In [101]:
def half_turn_ccw(arr, moves):
    for i in range(M//2):
        arr = apply_move['-r0'](arr, moves)
    
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['r0'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    
    for i in range(M//2):
        arr = apply_move['r0'](arr, moves)
    return arr

print_globe(half_turn(test_globe, []))

def rev_half_turn_cw(arr, moves):
    for i in range(M//2):
        arr = apply_move['-r0'](arr, moves)
    
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['-r0'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    
    for i in range(M//2):
        arr = apply_move['r0'](arr, moves)
    return arr

apply_move['half_turn'] = half_turn
apply_move['rev_half_turn'] = rev_half_turn

print_globe(rev_half_turn(test_globe, []))

[[ 1  2  3  4  5  6  7 23  8  9 10 11 12 13 14 15]
 [ 0 16 17 18 19 20 21 22 24 25 26 27 28 29 30 31]]
[[16  0  1  2  3  4  5  6  8  9 10 11 12 13 14 15]
 [17 18 19 20 21 22 23  7 24 25 26 27 28 29 30 31]]


In [102]:
test_globe = np.array([i for i in range(2*M)])

def top_right_cw(arr, moves):
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['-r0'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    return arr

def top_right_ccw(arr, moves):
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['r0'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    return arr

def top_left_ccw(arr, moves):
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['-r1'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    return arr

def top_left_cw(arr, moves):
    arr = apply_move['f0'](arr, moves)
    arr = apply_move['r1'](arr, moves)
    arr = apply_move['f0'](arr, moves)
    return arr

apply_move['top_right_cw'] = top_right_cw
apply_move['top_right_ccw'] = top_right_ccw
apply_move['top_left_cw'] = top_left_cw
apply_move['top_left_ccw'] = top_left_ccw

print_globe(top_right_cw(test_globe, []))
print_globe(top_right_ccw(test_globe, []))
print_globe(top_left_cw(test_globe, []))
print_globe(top_left_ccw(test_globe, []))

[[ 0  1  2  3  4  5  6  7 16  8  9 10 11 12 13 14]
 [17 18 19 20 21 22 23 15 24 25 26 27 28 29 30 31]]
[[ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 23]
 [ 8 16 17 18 19 20 21 22 24 25 26 27 28 29 30 31]]
[[24  0  1  2  3  4  5  6  8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23 25 26 27 28 29 30 31  7]]
[[ 1  2  3  4  5  6  7 31  8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23  0 24 25 26 27 28 29 30]]


## Phase 1: Sort Elements Into Top/Bottom Arrays

In [134]:
test_globe = puzzles_df[puzzles_df["puzzle_type"] == "globe_1/8"]['initial_state'][idx]
test_globe = np.array([int(x[1:]) for x in test_globe.split(';') if x])
print_globe(test_globe)

[[21  1  9 20 30  8 25 26 11  2 16 27  4 28  6 13]
 [17 31 12 18 19  0 29 10 22 14 24  7  3  5 15 23]]


In [135]:
moves = []

In [136]:
def is_sorted_top_bottom(arr):
    for i in range(M):
        if arr[i] >= M:
            return False
    for i in range(M, 2*M):
        if arr[i] < M:
            return False
    return True

def sort_top_bottom_helper(arr, moves):
    while not is_sorted_top_bottom(arr):
        
        rotate_locs = [
            (M//2, M + M // 2 - 1, 'top_right_ccw'),
            (M-1, M, 'top_right_cw'),
            (M//2-1, M + M//2, 'top_left_cw'),
            (0, 2*M-1, 'top_left_ccw')
        ]

        all_costs = []

        for a, b, op_name in rotate_locs:
            mn_a, mn_b = int(1e9), int(1e9)
            top_ops, bot_ops = [], []
            
            for i in range(M):
                if arr[i] >= M: # TODO: Try no abs. Can only r0
                    if abs(i - a) < mn_a:
                        mn_a = abs(i-a)

                        if i < a: # rotate right
                            top_ops = ['-r0'] * (a - i)
                        else:
                            top_ops = ['r0'] * (i - a)

            for i in range(M, 2*M):
                if arr[i] < M:
                    if abs(i - b) < mn_b:
                        mn_b = abs(i - b)

                        if i < b:
                            bot_ops = ['-r1'] * (b - i)
                        else:
                            bot_ops = ['r1'] * (i - b)

            all_costs.append((mn_a + mn_b, top_ops + bot_ops, op_name))

        all_costs = list(sorted(all_costs))
        for i in all_costs[0][1]:
            arr = apply_move[i](arr, moves)
        arr = apply_move[all_costs[0][2]](arr, moves)
    return arr

# print_globe(sort_top_bottom_helper(test_globe, moves))

In [137]:
def sort_top_bottom(arr, moves):
    a = []
    a_arr = sort_top_bottom_helper(arr, a)

    b = ['f0', 'f' + str(M//2)] # untested
    arr = apply_move['f0'](arr, [])
    
    for i in range(M//2):
        arr = apply_move['r0'](arr, [])
        arr = apply_move['r1'](arr, [])

    arr = apply_move['f0'](arr, [])

    for i in range(M//2):
        arr = apply_move['-r0'](arr, [])
        arr = apply_move['-r1'](arr, [])

    b_arr = sort_top_bottom_helper(arr, b)

    if len(b) < len(a):
        for i in b:
            moves.append(i)
        return b_arr

    for i in a:
        moves.append(i)
    return a_arr

test_globe = sort_top_bottom(test_globe, moves)
print_globe(test_globe)

[[ 4  8 14  9  1  6 13 10  2  0 11 12 15  5  3  7]
 [26 25 30 24 22 20 21 28 27 16 19 17 18 23 31 29]]


In [138]:
print(moves, len(moves))

['f0', 'f8', 'f0', 'r1', 'f0', 'f0', 'r1', 'f0', '-r1', 'f0', 'r0', 'f0', '-r0', '-r0', 'f0', 'r0', 'f0', '-r1', 'f0', '-r1', 'f0', '-r0', '-r0', '-r1', 'f0', 'r0', 'f0', '-r0', 'r1', 'f0', '-r0', 'f0', 'r1', 'r1', 'r1', 'r1', 'f0', 'r1', 'f0'] 39


In [242]:
def expand(path, n = 2, m = 16):    
    rows = [[] for _ in range(n)]
    
    def expand_f(x):
        for i in range(n):
            rows[i].append(x)
            rows[i].append('f0')
            rows[i].append(m - x)
    
    for i in path:
        if 'f' in i:
            if '-' in i:
                expand_f(int(i[2:]))
            else:
                expand_f(int(i[1:]))
        else:
            for j in range(n):
                temp = 'r' + str(j)

                if temp == i:
                    rows[j].append(1)
                    
                temp = '-r' + str(j)

                if temp == i:
                    rows[j].append(m-1)

    Rrows = [[] for _ in range(n)]

    for r in range(n):
        run = 0
        for i in rows[r]:
            if type(i) == type(0):
                run += i
            else:
                Rrows[r].append(run)
                Rrows[r].append('f0')
                run = 0
            
        Rrows[r].append(run)
    
    return Rrows

def shrink(rows, n = 2, m = 16):
    for i in range(len(rows[0])):
        mn = int(1e9)
        for j in range(n):
            if type(rows[j][i]) == type(1):
                rows[j][i] %= m
            else:
                mn = min(mn, rows[j][i-1])
        
        for j in range(n):  
            if type(rows[j][i]) == type('string'):
                rows[j][i-1] -= mn
                rows[j][i] = 'f' + str(mn)
                rows[j][i+1] += mn
    
    final_moves = []
    for i in range(len(rows[0])):
        if type(rows[0][i]) == type('string'):
            final_moves.append(rows[0][i])
            continue

        for j in range(n):
            if rows[j][i] > m/2:
                final_moves = final_moves + ['-r' + str(j)] * (m - rows[j][i])
            else:
                final_moves = final_moves + ['r' + str(j)] * (rows[j][i])

    final_final_moves = []
    i = 0
    while i < len(final_moves):
        if i != len(final_moves) - 1 and ('f' in final_moves[i]) and final_moves[i] == final_moves[i+1]:
            i += 2
        else:
            final_final_moves.append(final_moves[i])
            i += 1
    
    return final_final_moves
        

ex = expand(sample_submission['moves'][idx].split('.'), N, M)
res = shrink(ex, N, M)
res

['f17',
 'f5',
 'f16',
 'f8',
 'f11',
 'f20',
 'f9',
 'f18',
 'f8',
 'f3',
 'f0',
 'r1',
 'f4',
 'f20',
 'f22',
 'f7',
 'f19',
 'f8',
 'f1',
 'f7',
 'f13',
 'f3',
 'f22',
 'f2',
 'f8',
 'f4',
 'f15',
 'f7',
 'f16',
 'f11',
 'f26',
 'f29',
 'f3',
 'f7',
 'f1',
 'f12',
 'f10',
 'f19',
 'f8',
 'f17',
 'f26',
 'f31',
 'f5',
 'f21',
 'f19',
 'f11',
 'f13',
 'f24',
 'f29',
 'f6',
 '-r1',
 'f0',
 'f20',
 'f31',
 'f23',
 'f17',
 'f5',
 'f16',
 'f8',
 'f10',
 'f6',
 'f10',
 'f7',
 'f10',
 'f8',
 'f23',
 'f20',
 'f27',
 'f12',
 'r0',
 'f25',
 'f21',
 'r1',
 'f11',
 'r1',
 'f4',
 'f24',
 'f13',
 'f11',
 'f19',
 'f21',
 'f16',
 'f4',
 'f15',
 'f7',
 'f26',
 'f20',
 'f4',
 'f1',
 'f19',
 'f31',
 'r1',
 'f5',
 'f28',
 'f9',
 'r1',
 'f4',
 'f20',
 'f18',
 'f10',
 'f12',
 'f23',
 'f20',
 'f18',
 'f10',
 'f12',
 'f23',
 'f15',
 'f1',
 'f3',
 'f17',
 'f13',
 'f10',
 'f14',
 'f8',
 'f18',
 'f7',
 'f16',
 'f20',
 'f18',
 'f10',
 'f12',
 'f23',
 'f3',
 'r0',
 'f9',
 'r0',
 'f18',
 'r0',
 'f7',
 'r0',
 'f16

In [243]:
print(len(sample_submission['moves'][idx].split('.')))
print(len(res))

496
504


In [244]:
cur = np.array(puzzles['initial_state'][idx])
print(cur)
cur_type = puzzles['puzzle_type'][idx]

ALL = {}
temp = 0

calc = 0

for i in res:
    cur = cur[move_to_perm[cur_type][i]]

    if tuple(cur) in ALL:
        print(ALL[tuple(cur)], temp)
        calc += temp - ALL[tuple(cur)]

    if tuple(cur) not in ALL:
        ALL[tuple(cur)] = temp

    temp += 1

print('expected', np.array(puzzles['solution_state'][idx]))
print('got', cur)

['E' 'C' 'A' 'H' 'C' 'J' 'K' 'H' 'I' 'K' 'A' 'A' 'H' 'I' 'L' 'M' 'M' 'P'
 'I' 'N' 'C' 'L' 'M' 'K' 'G' 'O' 'F' 'D' 'J' 'N' 'F' 'B' 'J' 'G' 'P' 'L'
 'P' 'G' 'B' 'O' 'O' 'E' 'A' 'N' 'J' 'G' 'C' 'O' 'K' 'F' 'E' 'P' 'H' 'I'
 'N' 'M' 'E' 'F' 'D' 'L' 'B' 'D' 'D' 'B']
expected ['A' 'A' 'A' 'A' 'C' 'C' 'C' 'C' 'E' 'E' 'E' 'E' 'G' 'G' 'G' 'G' 'I' 'I'
 'I' 'I' 'K' 'K' 'K' 'K' 'M' 'M' 'M' 'M' 'O' 'O' 'O' 'O' 'B' 'B' 'B' 'B'
 'D' 'D' 'D' 'D' 'F' 'F' 'F' 'F' 'H' 'H' 'H' 'H' 'J' 'J' 'J' 'J' 'L' 'L'
 'L' 'L' 'N' 'N' 'N' 'N' 'P' 'P' 'P' 'P']
got ['A' 'A' 'A' 'A' 'C' 'C' 'C' 'C' 'E' 'E' 'E' 'E' 'G' 'G' 'G' 'G' 'I' 'I'
 'I' 'I' 'K' 'K' 'K' 'K' 'M' 'M' 'M' 'M' 'O' 'O' 'O' 'O' 'B' 'B' 'B' 'B'
 'D' 'D' 'D' 'D' 'F' 'F' 'F' 'F' 'H' 'H' 'H' 'H' 'J' 'J' 'J' 'J' 'L' 'L'
 'L' 'L' 'N' 'N' 'N' 'N' 'P' 'P' 'P' 'P']


In [231]:
print(len(res) - calc)

237


In [232]:
print(res[788:792])

[]


# Submissions #

The submission format requires moves in a sequence to be delimited by a period `.`. Indicate the inverse of a named move with a preceeding `-`. Moves are applied from left to right.

The `sample_submission.csv` contains a baseline solution for each puzzle.

In [12]:
sample_submission

,moves
id,
0,r1.-f1
1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...
393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....


Let's check that the given sequence for the first puzzle is actually a solution.

In [13]:
def apply_sequence(sequence, moves, state):
    """Apply a sequence of moves in array form to a color state."""
    state = np.asarray(state)
    for m in sequence.split('.'):
        state = state[moves[m]]
    return state


# Convert allowed_moves to dict and add inverse moves
all_moves = puzzle_info.loc[:, 'allowed_moves'].to_dict()
for ptype, moves in all_moves.copy().items():
    for m, arr in moves.copy().items():
        all_moves[ptype][f"-{m}"] = np.argsort(arr).tolist()


# Get info for the first puzzle
solution_state = puzzles.iloc[0, 1]
initial_state = puzzles.iloc[0, 2]
baseline_solution = sample_submission.loc[0, 'moves']

state = apply_sequence(baseline_solution, all_moves['cube_2/2/2'], initial_state)
np.array_equal(state, solution_state)

True

In this case, the resulting state is exactly equal to the required solution state. For puzzles with wildcards, there can be an allowed number of differences. See the competitions [Evaluation](https://www.kaggle.com/competitions/santa-2023/overview/evaluation) page for more info.

# Good luck! #

We hope this introduction was informative! Here are some places to follow up for more information:
- [The Complexity Dynamics of Magic Cubes and Twisty Puzzles](https://resplendence.org/dhushara.com/cubes/cubes.pdf)
- [Twisty Puzzle Museum](https://twistypuzzles.com/app/museum/museum_search.php)
- [Analyzing Rubik's Cube with GAP](https://www.gap-system.org/Doc/Examples/rubik.html) - You could try reproducing this with [SAGE](https://www.sagemath.org/), if you prefer.